# Algoritmo de búsqueda para selección de características

* **Javier Ruiz Muñiz**
* **Jesús Elías Rodríguez**

En este archivo se realizarán las implementaciones de los algoritmos SFS(Sequential forward selection) y SFFS(Sequential floating forward selection), siendo probadas y contrastando los resultados obtenidos para los dos conjuntos de datos proporcionados en la asignatura de IA.

Para la correcta implementación de los algoritmos citados, tuvimos que realizar un algoritmo de **validación robusta**, haciendo uso del framework de Python llamado Scikit-Learn, que nos proporcionó la función **cross_val_score** que nos permitió realizar validaciones mediante validación cruzada.

La implementación sería la siguiente:

In [3]:
def validacionRobusta(X,y,CV,hitRate):
    
    n_exp = 10  
  
    scoreI = list()
    accuracies = dict()

    for i in range(n_exp):
        tree = DecisionTreeClassifier()
        
        #Aplicamos validación cruzada
        scores = cross_val_score(tree,X,y,cv=CV,scoring=hitRate)
        #print(scores)
        avgScores = np.mean(scores)
        accuracies.update({i:avgScores})

    avgFinal = sum(accuracies.values())/n_exp
    return avgFinal

# SFS(Sequential forward selection)

En primer lugar, debemos sacar los elementos del fichero **CSV** e introducirlos en un **DataFrame** para poder trabajar con ellos de manera más eficiente. Además introduciremos las importaciones utilizadas para la realización de los algoritmos:

In [4]:
import pandas as pd
import numpy as np
import math
import sklearn

import matplotlib.pyplot as plt
from sklearn import tree, linear_model
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

##Sacamos los datos del fichero csv
datos = pd.read_csv('titanic.csv', sep=',', index_col=0)
dataFrame = pd.DataFrame(datos)

Una vez obtenidos los datos, podemos empezar a trabajar con ellos.

La implementación del algoritmo SFS es la siguiente:

In [6]:
def sfs(answerVar, predictorVar, D):
    CV = 3
    hitRate = 'balanced_accuracy'
    
    solucionActual = pd.DataFrame()
    
    supportDataFrame = predictorVar
   
    if D is None:
        D = len(predictorVar.columns)
     
    k=1
    while(k<D+1):
        
        lastValue = 0
        solucionTemporal = pd.DataFrame();
        
        for j in range(0, len(supportDataFrame.columns)):
            X = pd.concat([solucionActual,supportDataFrame.iloc[:,j]], axis=1)
            y = answerVar
            if k==1:
                validation = validacionRobusta(X.to_numpy().reshape(-1,1),y,CV,hitRate)
            else:
                validation = validacionRobusta(X,y,CV,hitRate)
                
            if(validation>lastValue):
                lastValue = validation
                solucionTemporal = X
        
        #Eliminamos del supportDataFrame la mejor variable
        bestColumn = solucionTemporal.columns[k-1]
        del supportDataFrame[bestColumn]
        
        
        #Actualizamos la solución actual; solucionActual = solucionActual + solucionTemporal
        solucionActual = solucionTemporal
        print(solucionActual)
        print(validation)
        
        k = k+1

    return solucionActual


Procederemos a continuación a realizar los test correspondientes:


In [7]:
answerVar = dataFrame.iloc[:,-1]
predictorVar = dataFrame.iloc[:,0:-1]
D=None

sfs(answerVar, predictorVar, D)

    Initial
3         0
1         1
3         2
1         1
3         0
..      ...
2         4
1         2
3         2
1         0
3         0

[891 rows x 1 columns]
0.6210574249832231
    Initial  SibSp
3         0      1
1         1      1
3         2      0
1         1      1
3         0      0
..      ...    ...
2         4      0
1         2      0
3         2      1
1         0      0
3         0      0

[891 rows x 2 columns]
0.7835538299300164
    Initial  SibSp  Deck
3         0      1     7
1         1      1     2
3         2      0     7
1         1      1     2
3         0      0     7
..      ...    ...   ...
2         4      0     7
1         2      0     1
3         2      1     7
1         0      0     2
3         0      0     7

[891 rows x 3 columns]
0.8053877864059056
    Initial  SibSp  Deck  Title
3         0      1     7     11
1         1      1     2     12
3         2      0     7      8
1         1      1     2     12
3         0      0     7     11
..     

,Initial,SibSp,Deck,Title,Is_Married,Sex,Alone,Fare_cat,Parch,Family_Size,Fare,Embarked,Age_band,Age
3,0,1,7,11,0,0,0,0,0,1,7.2500,0,1,22.0
1,1,1,2,12,1,1,0,3,0,1,71.2833,1,2,38.0
3,2,0,7,8,0,1,1,1,0,0,7.9250,0,1,26.0
1,1,1,2,12,1,1,0,3,0,1,53.1000,0,2,35.0
3,0,0,7,11,0,0,1,1,0,0,8.0500,0,2,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,4,0,7,14,0,0,1,1,0,0,13.0000,0,1,27.0
1,2,0,1,8,0,1,1,2,0,0,30.0000,0,1,19.0
3,2,1,7,8,0,1,0,2,2,3,23.4500,0,1,22.0
1,0,0,2,11,0,0,1,2,0,0,30.0000,1,1,26.0
